In [15]:
FILE = 'intersection_small'

In [16]:
class TNode:
    def __init__(self, id, lat, lng):
        self.id = id
        self.lat = lat
        self.lng = lng

class TGraph:
    node_id_to_index = {}
    nodes = []
    edges: list[set] = []
    edges_list: list[list] = []

    def __init__(self) -> None:
        self.node_id_to_index = {}
        self.nodes = []
        self.edges = []
        self.edges_list = []

    def add_node(self, node: TNode, ok_if_exists=True):
        if node.id in self.node_id_to_index:
            if not ok_if_exists:
                raise Exception("Node already exists")
            return
        index = len(self.nodes)
        self.node_id_to_index[node.id] = index
        self.nodes.append(node)
        self.edges.append(set())

    def add_edge(self, src_id, dst_id, ok_if_not_exists=True):
        if src_id not in self.node_id_to_index:
            if not ok_if_not_exists:
                raise Exception("Source node does not exist")
            return
        if dst_id not in self.node_id_to_index:
            if not ok_if_not_exists:
                raise Exception("Destination node does not exist")
            return
        
        src_index = self.node_id_to_index[src_id]
        dst_index = self.node_id_to_index[dst_id]
        self.edges[src_index].add(dst_index)
        self.edges[dst_index].add(src_index)
        self.edges_list.append([src_index, dst_index])

    def __str__(self) -> str:
        return f"{len(self.nodes)} nodes, {len(self.edges_list)} edges"

In [17]:
import json 

def read_graph_from_json(file_path: str) -> TGraph:
    with open(file_path, 'r') as file:
        data = json.load(file)
        graph = TGraph()
        for element in data['elements']:
            if element['type'] == 'node':
                graph.add_node(TNode(element['id'], element['lat'], element['lon']))
        
        for element in data['elements']:
            if element['type'] == 'way':
                for i in range(len(element['nodes']) - 1):
                    src = element['nodes'][i]
                    dst = element['nodes'][i + 1]
                    graph.add_edge(src, dst)

        return graph
    
graph = read_graph_from_json(f"../data_samples/osm_export/{FILE}.json")

print(graph)

2890 nodes, 3131 edges


In [18]:
from planegeometry.structures.segments import Segment, Point
from linesegmentintersections import bentley_ottman

segment_list = list()

MOVE = 0.000001

for src_index in range(len(graph.nodes)):
    src = graph.nodes[src_index]
    for dst_index in graph.edges[src_index]:
        if src_index >= dst_index:
            continue
            
        dst = graph.nodes[dst_index]
        
        # print(src.lat, src.lng, dst.lat, dst.lng)

        p1 = Point(src.lat, src.lng)
        p2 = Point(dst.lat, dst.lng)
        seg = Segment(p1, p2)
        
        # Need to move endpoints to center by a small amount.
        center = seg.center()
        dir1 = p1 - center
        new_p1 = p1 - dir1 * (1 / dir1.__abs__()) * MOVE

        dir2 = p2 - center
        new_p2 = p2 - dir2 * (1 / dir2.__abs__()) * MOVE

        segment_list.append([[new_p1.x, new_p1.y], [new_p2.x, new_p2.y]])

li = bentley_ottman(segment_list)

print(li)

[(43.23706200105764, 76.91400372352999), (43.23706436998761, 76.91368667694297), (43.2370842628245, 76.91358906536718), (43.23715256012122, 76.91375405635856), (43.24011677542051, 76.91545200573086), (43.240134202566516, 76.9152947341426), (43.240160500431735, 76.91531163707998), (43.24017168153925, 76.91548714658586), (43.24039571880108, 76.91563053426466), (43.24040074117031, 76.91546605167241)]


In [19]:
for i in range(len(li)):
    # i = list(inter)
    inter = li[i]
    print(inter.x, inter.y, i)

43.23706200105764 76.91400372352999 0
43.23706436998761 76.91368667694297 1
43.2370842628245 76.91358906536718 2
43.23715256012122 76.91375405635856 3
43.24011677542051 76.91545200573086 4
43.240134202566516 76.9152947341426 5
43.240160500431735 76.91531163707998 6
43.24017168153925 76.91548714658586 7
43.24039571880108 76.91563053426466 8
43.24040074117031 76.91546605167241 9


In [20]:
from planegeometry.structures.segments import Segment, Point

segment_list = list()

MOVE = 0.000001

edges_to_remove = []

for inter in li:
    inter_point = Point(inter.x, inter.y)
    for src_index in range(len(graph.nodes)):
        src = graph.nodes[src_index]
        for dst_index in graph.edges[src_index]:
            if src_index >= dst_index:
                continue
                
            dst = graph.nodes[dst_index]
            
            # print(src.lat, src.lng, dst.lat, dst.lng)

            p1 = Point(src.lat, src.lng)
            p2 = Point(dst.lat, dst.lng)
            seg = Segment(p1, p2)
            
            if min(p1.x, p2.x) <= inter_point.x and inter_point.x <= max(p1.x, p2.x) and (abs(seg.calculate_y(inter_point.x) - inter_point.y) < 0.00001):
                # print(seg, inter_point)
                edges_to_remove.append((src_index, dst_index))

for edge in edges_to_remove:
    src_index, dst_index = edge
    if dst_index in graph.edges[src_index]:
        graph.edges[src_index].remove(dst_index)
    if src_index in graph.edges[dst_index]:
        graph.edges[dst_index].remove(src_index)

In [21]:
# Format to graph to json format

data = {
    "elements": []
}

for node in graph.nodes:
    data['elements'].append({
        "type": "node",
        "id": node.id,
        "lat": node.lat,
        "lon": node.lng
    })

for src_index in range(len(graph.nodes)):
    for dst_index in graph.edges[src_index]:
        src = graph.nodes[src_index]
        dst = graph.nodes[dst_index]
        
        if src.id >= dst.id:
            continue
        
        data['elements'].append({
            "type": "way",
            "nodes": [src.id, dst.id]
        })

with open(f'../data_samples/prepared/{FILE}.json', 'w') as file:
    json.dump(data, file)